In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle
import json
import os

# # Сохранение в CSV файл
# combined_df.to_csv('Barotrauma_dataset_full.csv', index=False, encoding='utf-8')

# Загрузка объединенного датасета
df = pd.read_csv('Barotrauma_dataset_full.csv')

# Определяем mapping для группировки специализаций
SPECIALIZATION_MAPPING = {
    # Медик/Врач/Док группируем в Медик
    'медик': 'Медик', 'врач': 'Медик', 'док': 'Медик', 'доктор': 'Медик',
    'medic': 'Медик', 'doctor': 'Медик',
    
    # Капитан
    'капитан': 'Капитан', 'captain': 'Капитан',
    
    # Охранник
    'охранник': 'Охранник', 'security': 'Охранник', 'guard': 'Охранник',
    
    # Инженер
    'инженер': 'Инженер', 'engineer': 'Инженер',
    
    # Механик
    'механик': 'Механик', 'mechanic': 'Механик',
    
    # Помощник
    'помощник': 'Помощник', 'assistant': 'Помощник',
    
    # Ближайший бот
    'near': 'Near', 'ближайший': 'Near', 'бот': 'Near', 'bot': 'Near',
    
    # Общее обращение
    'all': 'All', 'все': 'All'
}

# Функция для нормализации специализации
def normalize_specialization(spec):
    spec_lower = str(spec).lower().strip()
    
    # Прямое соответствие
    if spec_lower in SPECIALIZATION_MAPPING:
        return SPECIALIZATION_MAPPING[spec_lower]
    
    # Поиск вхождений в тексте
    for key, value in SPECIALIZATION_MAPPING.items():
        if key in spec_lower:
            return value
    
    # Если не нашли - возвращаем оригинал (или можно задать значение по умолчанию)
    return 'All'

# Применяем нормализацию
df['normalized_specialization'] = df['specialization'].apply(normalize_specialization)

# Проверяем распределение
print("Распределение по классам:")
print(df['normalized_specialization'].value_counts())

def remove_addressing(text, specialization):
    # Создаем список всех синонимов для данной специализации
    synonyms = []
    for key, value in SPECIALIZATION_MAPPING.items():
        if value == specialization:
            synonyms.append(key)
    
    # Добавляем русские и английские варианты
    additional_synonyms = {
        'Медик': ['медик', 'врач', 'док', 'доктор', 'medic', 'doctor'],
        'Капитан': ['капитан', 'captain'],
        'Охранник': ['охранник', 'security', 'guard'],
        'Инженер': ['инженер', 'engineer'],
        'Механик': ['механик', 'mechanic'],
        'Помощник': ['помощник', 'assistant'],
        'Near': ['near', 'ближайший', 'бот', 'bot'],
        'All': ['all', 'все']
    }
    
    if specialization in additional_synonyms:
        synonyms.extend(additional_synonyms[specialization])
    
    # Удаляем дубликаты
    synonyms = list(set(synonyms))
    
    cleaned_text = str(text)
    
    # Удаляем каждое возможное обращение
    for synonym in synonyms:
        # Паттерны для удаления (с учетом разных форм)
        patterns = [
            rf'\b{re.escape(synonym)}\b[\s,]*',
            rf'\b{re.escape(synonym.capitalize())}\b[\s,]*',
        ]
        
        for pattern in patterns:
            cleaned_text = re.sub(pattern, '', cleaned_text, flags=re.IGNORECASE)
    
    # Удаляем лишние пробелы и запятые
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    cleaned_text = re.sub(r'^\s*,\s*', '', cleaned_text)  # Удаляем запятую в начале
    cleaned_text = re.sub(r'\s*,\s*$', '', cleaned_text)  # Удаляем запятую в конце
    
    return cleaned_text

# Применяем функцию ко всему датасету
df['cleaned_text'] = df.apply(
    lambda row: remove_addressing(row['text'], row['normalized_specialization']), 
    axis=1
)

# Фиксируем 8 классов
FINAL_CLASSES = ['Капитан', 'Охранник', 'Медик', 'Инженер', 'Механик', 'Помощник', 'Near', 'All']

# Кодируем специализации
label_encoder = LabelEncoder()
label_encoder.fit(FINAL_CLASSES)  # Фиксируем классы
df['specialization_encoded'] = label_encoder.transform(df['normalized_specialization'])

# Параметры для текста
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 100

# Создаем улучшенный токенизатор для PyTorch
class TextTokenizer:
    def __init__(self, max_vocab_size, oov_token="<OOV>"):
        self.max_vocab_size = max_vocab_size
        self.oov_token = oov_token
        self.word_index = {}
        self.index_word = {}
        self.vocab_size = 0
        
    def fit_on_texts(self, texts):
        word_counts = {}
        for text in texts:
            # Более качественная токенизация с учетом пунктуации
            words = re.findall(r'\b\w+\b', str(text).lower())
            for word in words:
                word_counts[word] = word_counts.get(word, 0) + 1
        
        # Сортируем слова по частоте
        sorted_words = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
        
        # Создаем словарь
        self.word_index = {self.oov_token: 0}
        self.index_word = {0: self.oov_token}
        
        idx = 1
        for word, count in sorted_words:
            if idx < self.max_vocab_size:
                self.word_index[word] = idx
                self.index_word[idx] = word
                idx += 1
            else:
                break
                
        self.vocab_size = len(self.word_index)
        print(f"Создан словарь размером: {self.vocab_size} слов")
        
    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            words = re.findall(r'\b\w+\b', str(text).lower())
            sequence = []
            for word in words:
                sequence.append(self.word_index.get(word, 0))  # 0 для OOV
            sequences.append(sequence)
        return sequences

# Инициализируем и обучаем токенизатор
tokenizer = TextTokenizer(MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df['text'])

# Преобразуем текст в последовательности
X_sequences = tokenizer.texts_to_sequences(df['text'])

# Функция для padding последовательностей
def pad_sequences(sequences, maxlen, padding='post', truncating='post', value=0):
    result = []
    for seq in sequences:
        if len(seq) > maxlen:
            if truncating == 'post':
                seq = seq[:maxlen]
            else:
                seq = seq[-maxlen:]
        else:
            pad_length = maxlen - len(seq)
            if padding == 'post':
                seq = seq + [value] * pad_length
            else:
                seq = [value] * pad_length + seq
        result.append(seq)
    return np.array(result)

X_padded = pad_sequences(X_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Подготовка меток
y = df['specialization_encoded'].values

# Проверяем баланс классов
print("\nРаспределение классов:")
for i, class_name in enumerate(FINAL_CLASSES):
    count = np.sum(y == i)
    print(f"{class_name}: {count} samples ({count/len(y)*100:.2f}%)")

# Разделение на train/test
X_train, X_test, y_train, y_test, text_train, text_test = train_test_split(
    X_padded, y, df['text'], test_size=0.2, random_state=42, stratify=y
)

print(f"Размер тренировочной выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")
print(f"Количество классов: {len(FINAL_CLASSES)}")
print(f"Размер словаря: {tokenizer.vocab_size}")

# Создаем улучшенную PyTorch модель
class SpecializationClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers=2, dropout=0.3):
        super(SpecializationClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                           dropout=dropout, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)  # *2 из-за bidirectional
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.relu = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(hidden_dim)
        self.batch_norm2 = nn.BatchNorm1d(hidden_dim // 2)
        
    def forward(self, x):
        embedded = self.embedding(x)
        
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # Используем последний hidden state из bidirectional LSTM
        hidden_forward = hidden[-2, :, :]  # forward direction
        hidden_backward = hidden[-1, :, :]  # backward direction
        out = torch.cat((hidden_forward, hidden_backward), dim=1)
        
        out = self.dropout(out)
        out = self.relu(self.fc1(out))
        out = self.batch_norm1(out)
        out = self.relu(self.fc2(out))
        out = self.batch_norm2(out)
        out = self.dropout(out)
        out = self.fc3(out)
        
        return out

# Параметры модели
HIDDEN_DIM = 128
NUM_LAYERS = 2
DROPOUT = 0.3
NUM_CLASSES = len(FINAL_CLASSES)
LEARNING_RATE = 0.001

# Создаем модель
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

model = SpecializationClassifier(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=NUM_CLASSES,
    n_layers=NUM_LAYERS,
    dropout=DROPOUT
).to(device)

print(f"Модель создана: {sum(p.numel() for p in model.parameters()):,} параметров")

# Функция для создания DataLoader
def create_dataloader(X, y, batch_size=32, shuffle=True):
    dataset = TensorDataset(
        torch.LongTensor(X),
        torch.LongTensor(y)
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Создаем DataLoader'ы
train_loader = create_dataloader(X_train, y_train, batch_size=32)
test_loader = create_dataloader(X_test, y_test, batch_size=32, shuffle=False)

# Функция для вычисления accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

# Функция для обучения
def train_model(model, train_loader, test_loader, epochs=20, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    train_losses = []
    train_accuracies = []
    val_accuracies = []
    
    best_accuracy = 0.0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_accuracy = 0
        batches = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # Gradient clipping для стабильности
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(outputs, batch_y)
            batches += 1
        
        train_accuracy = total_accuracy / batches
        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)
        
        # Валидация
        val_accuracy = evaluate_model(model, test_loader)
        val_accuracies.append(val_accuracy)
        
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        
        print(f'Epoch [{epoch+1}/{epochs}], LR: {current_lr:.6f}, Loss: {avg_loss:.4f}, '
              f'Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%')
        
        # Сохраняем лучшую модель
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_accuracy': best_accuracy,
                'train_losses': train_losses,
                'train_accuracies': train_accuracies,
                'val_accuracies': val_accuracies
            }, 'best_specialization_model_8classes.pth')
            print(f'Новая лучшая модель сохранена с точностью: {best_accuracy:.2f}%')
    
    return train_losses, train_accuracies, val_accuracies

# Функция для оценки модели
def evaluate_model(model, test_loader):
    model.eval()
    total_accuracy = 0
    batches = 0
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            accuracy = calculate_accuracy(outputs, batch_y)
            total_accuracy += accuracy
            batches += 1
    
    return total_accuracy / batches

# Обучаем модель
print("\nНачало обучения...")
train_losses, train_accuracies, val_accuracies = train_model(
    model, train_loader, test_loader, epochs=30, lr=LEARNING_RATE
)

# Функция для сохранения всех необходимых файлов
def save_model_artifacts(model, tokenizer, label_encoder, special_mapping, final_classes, max_sequence_length):
    """Сохраняет все необходимые артефакты для использования модели"""
    
    # Создаем папку для модели
    model_dir = "specialization_model_package"
    os.makedirs(model_dir, exist_ok=True)
    
    # 1. Сохраняем веса модели
    torch.save(model.state_dict(), f"{model_dir}/model_weights.pth")
    
    # 2. Сохраняем архитектуру модели и параметры
    model_config = {
        'vocab_size': tokenizer.vocab_size,
        'embedding_dim': EMBEDDING_DIM,
        'hidden_dim': HIDDEN_DIM,
        'output_dim': len(final_classes),
        'n_layers': NUM_LAYERS,
        'dropout': DROPOUT,
        'max_sequence_length': max_sequence_length,
        'bidirectional': True
    }
    
    with open(f"{model_dir}/model_config.json", 'w', encoding='utf-8') as f:
        json.dump(model_config, f, ensure_ascii=False, indent=2)
    
    # 3. Сохраняем токенизатор
    with open(f"{model_dir}/tokenizer.pkl", 'wb') as f:
        pickle.dump(tokenizer, f)
    
    # 4. Сохраняем label encoder
    with open(f"{model_dir}/label_encoder.pkl", 'wb') as f:
        pickle.dump(label_encoder, f)
    
    # 5. Сохраняем mapping специализаций
    with open(f"{model_dir}/specialization_mapping.json", 'w', encoding='utf-8') as f:
        json.dump(special_mapping, f, ensure_ascii=False, indent=2)
    
    # 6. Сохраняем финальные классы
    with open(f"{model_dir}/final_classes.json", 'w', encoding='utf-8') as f:
        json.dump(final_classes, f, ensure_ascii=False, indent=2)
    
    print(f"Все файлы модели сохранены в папку: {model_dir}")

# Сохраняем все артефакты модели
save_model_artifacts(model, tokenizer, label_encoder, SPECIALIZATION_MAPPING, FINAL_CLASSES, MAX_SEQUENCE_LENGTH)

# Загружаем лучшую модель для предсказаний
checkpoint = torch.load('best_specialization_model_8classes.pth', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"\nЛучшая точность на валидации: {checkpoint['best_accuracy']:.2f}%")

def predict_and_clean_text(model, tokenizer, label_encoder, input_texts):
    """
    Предсказывает специализацию и очищает текст от обращений
    """
    # Преобразуем текст в последовательности
    sequences = tokenizer.texts_to_sequences(input_texts)
    padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    
    # Предсказываем
    with torch.no_grad():
        inputs = torch.LongTensor(padded_sequences).to(device)
        outputs = model(inputs)
        probabilities = torch.softmax(outputs, dim=1)
        predicted_classes = torch.argmax(outputs, dim=1).cpu().numpy()
    
    predicted_specializations = label_encoder.inverse_transform(predicted_classes)
    
    # Очищаем текст от обращений
    cleaned_texts = []
    for text, spec in zip(input_texts, predicted_specializations):
        cleaned_text = remove_addressing(text, spec)
        cleaned_texts.append(cleaned_text)
    
    return predicted_specializations, cleaned_texts

# Предсказываем для всего датасета
all_texts = df['text'].tolist()
predicted_specs, cleaned_texts = predict_and_clean_text(model, tokenizer, label_encoder, all_texts)

# Вычисляем accuracy на всем датасете
true_labels = df['normalized_specialization'].values
accuracy = np.mean(predicted_specs == true_labels) * 100
print(f"\nТочность на всем датасете: {accuracy:.2f}%")

# Создаем финальный датасет
result_df = pd.DataFrame({
    'original_text': df['text'],
    'true_specialization': df['normalized_specialization'],
    'predicted_specialization': predicted_specs,
    'cleaned_text': cleaned_texts,
    'is_correct': (predicted_specs == true_labels)
})

# Сохраняем результат
result_df.to_csv('predicted_specializations_8classes_pytorch.csv', index=False, encoding='utf-8')

print("Файл 'predicted_specializations_8classes_pytorch.csv' успешно создан!")
print(f"Всего обработано: {len(result_df)} текстов")
print(f"Используемые классы: {FINAL_CLASSES}")

# Тестируем на различных обращениях
test_texts = [
    "Доктор, проверь пациента",
    "Врач, подойди сюда", 
    "Медик, нужна помощь",
    "Механик, почини двигатель",
    "Инженер, проверь системы",
    "Капитан, доложите ситуацию",
    "Ближайший бот, иди сюда",
    "Все, внимание!",
    "Near, come here",
    "All hands on deck!"
]

test_specs, test_cleaned = predict_and_clean_text(model, tokenizer, label_encoder, test_texts)

test_results = pd.DataFrame({
    'input_text': test_texts,
    'predicted_specialization': test_specs,
    'cleaned_text': test_cleaned
})

print("\nТестовые примеры:")
print(test_results)

print("\n" + "="*50)
print("Ключевые улучшения:")
print("="*50)
print("1. Bidirectional LSTM для лучшего понимания контекста")
print("2. Batch Normalization для стабильности обучения")
print("3. Gradient Clipping для предотвращения взрыва градиентов")
print("4. Learning Rate Scheduling")
print("5. Weight Decay для регуляризации")
print("6. Улучшенная токенизация с учетом пунктуации")
print("7. Стратифицированное разделение данных")